# Interferogram Processing with ISCE topsApp

In [18]:
# Import required packages
import logging
log = logging.getLogger()
log.setLevel(logging.WARN)
import os
import getpass
import asf_search as asf
import isce
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [4]:
# Set environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)
print(os.environ['PATH'])

/home/jovyan/.local/ARIA-tools/tools/bin:/home/jovyan/.local/ARIA-tools/tools/ARIAtools:/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications:/home/jovyan/.local/envs/insar_analysis/bin:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/bin:/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/bin:/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications


In [5]:
!which topsApp.py

/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications/topsApp.py


In [6]:
def plotcomplexdata(GDALfilename,
                    title=None, aspect=1,
                    datamin=None, datamax=None,
                    interpolation='nearest',
                    draw_colorbar=None, colorbar_orientation="horizontal"):
    # Load the data into numpy array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    slc = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+slc.shape[1]*deltax
    lasty = firsty+slc.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    try:
        slc[slc==0]=np.nan
    except:
        pass
    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(slc), vmin = datamin, vmax=datamax,
                   cmap='gray', extent=[xmin,xmax,ymin,ymax],
                   interpolation=interpolation)
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(slc), cmap='rainbow',
                    vmin=-np.pi, vmax=np.pi,
                    extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2, orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show()
    
    # clearing the data
    slc = None
    
def plotdata(GDALfilename, band=1,
             title=None,colormap='gray',
             aspect=1, background=None,
             datamin=None, datamax=None,
             interpolation='nearest',
             nodata = None,
             draw_colorbar=True, colorbar_orientation="horizontal"):
    
    # Read the data into an array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(band).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    try:
        if nodata is not None:
            data[data == nodata] = np.nan
    except:
        pass
        
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+data.shape[1]*deltax
    lasty = firsty+data.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    if background is None:
        try:
            data[data==0]=np.nan
        except:
            pass
        
        fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show()
    
    # clearing the data
    data = None

In [6]:
# make required directories
dirs = ['/home/jovyan/rmnp_landslide/proc/orbital', '/home/jovyan/rmnp_landslide/proc/SLC']
for i in dirs:
    if not os.path.exists(i):
        os.makedirs(i)

## Define desired pairs from list

In [8]:
scene_list = ['S1B_IW_SLC__1SDV_20200717T005351_20200717T005418_022504_02AB6A_333A',
              'S1B_IW_SLC__1SDV_20200717T005351_20200717T005418_022504_02AB6A_333A',
              'S1B_IW_SLC__1SDV_20200722T010143_20200722T010210_022577_02AD97_0AF9',
              'S1B_IW_SLC__1SDV_20200722T010208_20200722T010235_022577_02AD97_AE1D',
              'S1B_IW_SLC__1SDV_20200729T005352_20200729T005419_022679_02B0C0_8427',
              'S1B_IW_SLC__1SDV_20200803T010144_20200803T010211_022752_02B2E3_2F86',
              'S1B_IW_SLC__1SDV_20200803T010209_20200803T010236_022752_02B2E3_C178',
              'S1B_IW_SLC__1SDV_20200810T005352_20200810T005419_022854_02B621_D8F8',
              'S1B_IW_SLC__1SDV_20200815T010145_20200815T010212_022927_02B84B_AD30',
              'S1B_IW_SLC__1SDV_20200815T010210_20200815T010237_022927_02B84B_FCB6',
              'S1B_IW_SLC__1SDV_20200822T005353_20200822T005420_023029_02BB99_67C5',
              'S1B_IW_SLC__1SDV_20200827T010146_20200827T010213_023102_02BDCC_5F7D',
              'S1B_IW_SLC__1SDV_20200827T010211_20200827T010238_023102_02BDCC_BB77',
              'S1B_IW_SLC__1SDV_20200903T005354_20200903T005421_023204_02C10B_ABD8',
              'S1B_IW_SLC__1SDV_20200908T010146_20200908T010213_023277_02C345_9DE5',
              'S1B_IW_SLC__1SDV_20200908T010211_20200908T010238_023277_02C345_0255'
             ]

In [31]:
def select_pairs(scene_list, max_temp_bline):
    scene_dates = {}
    for scene in scene_list:
        date = scene[17:25]
        scene_dates[date] = scene
        
    int_dict = {}
    for date1 in scene_dates:
        for date2 in scene_dates:
            if datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=max_temp_bline) and not date1 >= date2 :
                int_dict[f'{date1}-{date2}'] = [scene_dates[date1], scene_dates[date2]]
    
    return int_dict

In [33]:
select_pairs(scene_list, 12)

{'20200717-20200722': ['S1B_IW_SLC__1SDV_20200717T005351_20200717T005418_022504_02AB6A_333A',
  'S1B_IW_SLC__1SDV_20200722T010208_20200722T010235_022577_02AD97_AE1D'],
 '20200722-20200729': ['S1B_IW_SLC__1SDV_20200722T010208_20200722T010235_022577_02AD97_AE1D',
  'S1B_IW_SLC__1SDV_20200729T005352_20200729T005419_022679_02B0C0_8427'],
 '20200729-20200803': ['S1B_IW_SLC__1SDV_20200729T005352_20200729T005419_022679_02B0C0_8427',
  'S1B_IW_SLC__1SDV_20200803T010209_20200803T010236_022752_02B2E3_C178'],
 '20200803-20200810': ['S1B_IW_SLC__1SDV_20200803T010209_20200803T010236_022752_02B2E3_C178',
  'S1B_IW_SLC__1SDV_20200810T005352_20200810T005419_022854_02B621_D8F8'],
 '20200810-20200815': ['S1B_IW_SLC__1SDV_20200810T005352_20200810T005419_022854_02B621_D8F8',
  'S1B_IW_SLC__1SDV_20200815T010210_20200815T010237_022927_02B84B_FCB6'],
 '20200815-20200822': ['S1B_IW_SLC__1SDV_20200815T010210_20200815T010237_022927_02B84B_FCB6',
  'S1B_IW_SLC__1SDV_20200822T005353_20200822T005420_023029_02BB99_

## Download SLCs

In [7]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

 ········


In [8]:
# Change to SLC directory 
os.chdir('/home/jovyan/rmnp_landslide/proc/SLC')

logging.getLogger("urllib3").setLevel(logging.WARNING)

reference = 'S1B_IW_SLC__1SDV_20210822T010216_20210822T010243_028352_036202_417A'
secondary = 'S1B_IW_SLC__1SDV_20210903T010217_20210903T010244_028527_036777_F591'
granules = [reference, secondary]

In [9]:
%%time

results = asf.granule_search(granules)
session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path='/home/jovyan/rmnp_landslide/proc/SLC', processes=2, session=session)

CPU times: user 57.7 ms, sys: 20.6 ms, total: 78.4 ms
Wall time: 2min 7s


In [10]:
# Create a list of zipped SLCs. Assume there are no other files with .zip extension in SLC directory
datadir = listdir('/home/jovyan/rmnp_landslide/proc/SLC')

scenes = []
for file in datadir:
    if file[-4:]=='.zip':
        scenes.append(file[:-4])
        
print(scenes)

['S1B_IW_SLC__1SDV_20210903T010217_20210903T010244_028527_036777_F591', 'S1B_IW_SLC__1SDV_20210822T010216_20210822T010243_028352_036202_417A']


## Get orbital files

In [11]:
os.chdir('/home/jovyan/rmnp_landslide/proc/orbital')

In [12]:
%%bash

wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

--2022-09-06 16:22:01--  https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5054 (4.9K) [text/plain]
Saving to: ‘fetchOrbit.py’

     0K ....                                                  100% 71.1M=0s

2022-09-06 16:22:01 (71.1 MB/s) - ‘fetchOrbit.py’ saved [5054/5054]



In [13]:
# grab orbital files with fetchOrbit.py
for file in scenes:
    os.system(f'./fetchOrbit.py -i {file}')

Reference time:  2021-09-03 01:02:44
Satellite name:  S1B
Reference time:  2021-08-22 01:02:43
Satellite name:  S1B


## Write input file

In [14]:
os.chdir('/home/jovyan/rmnp_landslide/proc/')

In [15]:
with open('topsApp.xml', 'w') as f:
    f.write(f"""<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
  <component name="topsinsar">
    <property name="Sensor name">SENTINEL1</property>
    <component name="reference">
        <property name="orbit directory">/home/jovyan/rmnp_landslide/proc/orbital</property>
        <property name="output directory">reference</property>
        <property name="safe">/home/jovyan/rmnp_landslide/proc/SLC/{reference}.zip</property>
    </component>
    <component name="secondary">
        <property name="orbit directory">/home/jovyan/rmnp_landslide/proc/orbital</property>
        <property name="output directory">secondary</property>
        <property name="safe">/home/jovyan/rmnp_landslide/proc/SLC/{secondary}.zip</property>
    </component>
    <property name="demfilename">/home/jovyan/rmnp_landslide/proc/usgs_10m.dem.wgs84</property>
    <property name="useGPU">True</property>
    <property name="swaths">[3]</property>
    <property name="range looks">6</property>
    <property name="azimuth looks">1</property>
    <property name="region of interest">[40.295, 40.310, -105.69, -105.67]</property>
    <property name="do esd">False</property>
    <property name="do ionospherecorrection">False</property>
    <property name="do unwrap">True</property>
    <property name="unwrapper name">snaphu_mcf</property>
    <property name="do denseoffsets">False</property>
    <property name="geocode demfilename">/home/jovyan/rmnp_landslide/proc/usgs_10m.dem.wgs84</property>
    <property name="geocode list">['merged/topophase.cor', 'merged/filt_topophase.unw', 'merged/filt_topophase.flat']</property>
  </component>
</topsApp>""")

## topsApp processing

In [16]:
%%time

!topsApp.py --start=preprocess --end=verifyDEM

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-06 16:22:19,738 - isce.insar - INFO - ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
Step processing
Cannot open PICKLE/startup
Running step preprocess
Input XML files:  ['/vsizip//home/jovyan/rmnp_landslide/proc/SLC/S1B_IW_SLC__1SDV_20210822T010216_20210822T010243_028352_036202_417A.zip/S1B_IW_SLC__1SDV_20210822T010216_20210822T010243_028352_036202_417A.SAFE/annotation/s1b-iw3-slc-vv-20210822t010216-20210822t010241-028352-0

In [17]:
%%time

!topsApp.py --dostep=topo

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-06 16:22:42,663 - isce.insar - INFO - ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
Step processing
Running step topo
2022-09-06 16:22:42,774 - isce.zerodop.topozero - WARNING - Default Peg heading set to: -0.23633792544467572
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 1 
901706	2.32956	
GDAL open (R): usgs_10m.dem.wgs84.vrt
API open (WR): geom_reference/IW3/lat_01.rdr
API open (WR): geom_reference/IW3/lon_

In [18]:
%%time

!topsApp.py --start=subsetoverlaps --end=mergebursts

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-06 16:24:35,297 - isce.insar - INFO - ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
Step processing
Running step subsetoverlaps
Dumping the application's pickle object _insar to file  PICKLE/subsetoverlaps
The remaining steps are (in order):  ['coarseoffsets', 'coarseresamp', 'overlapifg', 'prepesd', 'esd', 'rangecoreg', 'fineoffsets', 'fineresamp', 'ion', 'burstifg', 'mergebursts', 'filter', 'unwrap', 'unwrap2stage', 'geoc

In [19]:
%%time

!topsApp.py --start=filter --end=unwrap2stage

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-06 16:26:07,865 - isce.insar - INFO - ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
Step processing
Running step filter
2022-09-06 16:26:07,911 - isce.topsinsar.runFilter - INFO - Applying power-spectral filter
GDAL open (R): merged/topophase.flat.vrt
API open (WR): merged/filt_topophase.flat
2022-09-06 16:26:07,928 - isce.mroipac.filter - DEBUG - width: 4281
2022-09-06 16:26:07,929 - isce.mroipac.filter - DEBUG - length: 1

In [20]:
!topsApp.py --dostep=geocode

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-06 16:30:45,354 - isce.insar - INFO - ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.1, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20220811, CURRENT_SVN_REVISION = 
Step processing
Running step geocode
2022-09-06 16:30:45,404 - isce.topsinsar.runGeocode - INFO - Geocoding Image
Number of products to geocode:  3
Output:  merged/topophase.cor.geo
API open (WR): merged/dem.crop
API open (WR): merged/topophase.cor.geo
GDAL open (R): merged/topophase.cor.vrt
GDAL open (R): usgs_10m.dem.wgs84.vrt
 Using neare

## extract wrapped phase from complex values

In [21]:
os.chdir('/home/jovyan/rmnp_landslide/proc/merged/')

In [22]:
!isce2gis.py vrt -i filt_topophase.flat.geo

!gdal_calc.py --type Float32 -A filt_topophase.flat.geo.vrt --calc="numpy.angle(A)" --outfile=filt_topophase.flat.geo.phs.tif --NoDataValue=0.0 --overwrite

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for merged/filt_topophase.flat.geo
Traceback (most recent call last):
  File "/home/jovyan/.local/envs/insar_analysis/lib/python3.8/xml/etree/ElementTree.py", line 786, in _get_writer
    write = file_or_filename.write
AttributeError: 'str' object has no attribute 'write'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/isce/applications/isce2gis.py", line 88, in <module>
    isce2vrt(inps.infile)
  File "/home/jovyan/.